In [1]:
root_dir = '/home2/glee/dissertation/1_tech_gen_impact/class2class/Tech_Gen/'
master_dir = '/home2/glee/dissertation/1_tech_gen_impact/master/Tech_Gen/'
import sys
sys.path.append(root_dir)

import copy
import gc
import os
import argparse
import math
import time
import pickle
import re
import multiprocess as mp
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
sys.path.append("/share/tml_package")
from tml import utils
from scipy import io
from tqdm import tqdm
from collections import OrderedDict

import torch
from torch.nn import functional as F
from torch.nn import DataParallel as DP
from torch.utils.data import TensorDataset, DataLoader, Subset, Dataset
from accelerate import Accelerator
import pytorch_model_summary

import optuna
from optuna.samplers import RandomSampler, TPESampler
from optuna.integration import SkoptSampler

import numpy as np
import pandas as pd
import scipy.stats
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import matthews_corrcoef, precision_recall_fscore_support, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

from data import TechDataset, CVSampler
from models import Transformer, Predictor
from train_utils import EarlyStopping, perf_eval, objective_cv, build_model, train_model, validate_model_mp
from utils import token2class, DotDict, to_device

from cleantext.sklearn import CleanTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# PART 1: Configuration

In [2]:
args = argparse.Namespace(
    data_type="class+claim",
    data_file = "collection_[H01L,H10][2017].csv",
    target_ipc="H01L",
    pred_type="classification",
    n_TC = 5,
    use_pretrained_tokenizer=False,
    do_train=None,
    do_tune=None,
    n_folds=None,
    batch_size=16,
    max_epochs=50,
    use_accelerator=None,
    do_save=False,
    n_gpus=4,
    light=True,
#     config_file=os.path.join(root_dir, "configs", "USED_configs", "[CONFIGS]2023-04-11_00:39.json"),
    config_file=None,
    eval_train_set=False)

data_dir = os.path.join(master_dir, "data")
model_dir = os.path.join(root_dir, "models")
result_dir = os.path.join(root_dir, "results")
config_dir = os.path.join(root_dir, "configs")

# args = parser.parse_args()
if args.config_file is not None:
    config_file = args.config_file
else:
    config_file = os.path.join(config_dir, "configs_light.json") if args.light else os.path.join(config_dir, "configs.json")
configs = DotDict().load(config_file)
org_config_keys = {key: list(configs[key].keys()) for key in configs.keys()}

instant_configs = {key: value for (key, value) in vars(args).items() if value is not None} # if any argument passed when main.py executed
instant_configs_for_update = {configkey: {key: value for (key,value) in instant_configs.items() if key in org_config_keys[configkey]} for configkey in org_config_keys.keys()}
for key, value in configs.items():
    value.update(instant_configs_for_update[key])

regex_ipc = re.compile('[A-Z](?![\\D])')
if regex_ipc.match(configs.data.target_ipc) is None:
    configs.data.update({"target_ipc": "ALL"})
elif len(configs.data.target_ipc) > 5:
    configs.data.update({"target_ipc": configs.data.target_ipc[:4]})

if configs.model.model_type == "enc-pred-dec":
    configs.train.loss_weights["recon"] = configs.train.loss_weights["recon"] / sum(configs.train.loss_weights.values())
    configs.train.loss_weights["y"] = 1 - configs.train.loss_weights["recon"]
elif configs.model.model_type == "enc-pred":
    configs.train.loss_weights = {"recon": 0, "y": 1}
elif configs.model.model_type == "enc-dec":
    configs.train.loss_weights = {"recon": 1, "y": 0}

if configs.train.use_accelerator:
    accelerator = Accelerator()
    device_ids = list(range(torch.cuda.device_count()))
    device = accelerator.device
    configs.train.update({"accelerator": accelerator})
else:
    if torch.cuda.is_available():
        device_ids = list(range(torch.cuda.device_count()))
        gpu_usages = [np.sum([float(usage.split("uses")[-1].replace(" ","").replace("MB","")) for usage in torch.cuda.list_gpu_processes(id).split("GPU memory") if not usage=="" and "no processes are running" not in usage]) for id in device_ids]
        device_ids = np.argsort(gpu_usages)[:configs.train.n_gpus]
        device_ids = list(map(lambda x: torch.device('cuda', x),list(device_ids)))
        device = device_ids[0] # main device
        torch.cuda.set_device(device)
    else:
        device = torch.device('cpu')
        device_ids = []

configs.data.update({"root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "result_dir": result_dir,
                        "pretrained_enc": configs.model.pretrained_enc,
                        "pretrained_dec": configs.model.pretrained_dec,
                        "data_nrows": None})
configs.train.update({"device": device,
                        "device_ids": device_ids,
                        "root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "use_keywords": configs.data.use_keywords,
                        "early_stop_patience": int(0.3*configs.train.max_epochs)})
configs.model.update({"device": device,
                        "device_ids": device_ids,
                        "n_directions": 2 if configs.model.bidirec else 1,
                        "use_accelerator": configs.train.use_accelerator})

## Set hyperparameters for model training (To be TUNED)
if configs.train.do_train and configs.train.do_tune:
    n_layers = configs.model.n_layers = None
    d_embedding = configs.model.d_embedding = None
    d_enc_hidden = configs.model.d_enc_hidden = None
    d_pred_hidden = configs.model.d_pred_hidden = None
    d_latent = 64
    learning_rate = configs.train.learning_rate = None
    batch_size = configs.train.batch_size = None
    config_name = "HPARAM_TUNING"
    final_model_path = None
else:
    n_layers = configs.model.n_layers
    d_embedding = configs.model.d_embedding
    d_enc_hidden = configs.model.d_enc_hidden
    d_pred_hidden = configs.model.d_pred_hidden
    d_latent = configs.model.d_enc_hidden * configs.model.n_directions
    d_latent = 64

    configs.model.update({"d_latent": d_latent})

    key_components = {"data": ["target_ipc", "vocab_size"], "model": ["n_layers", "d_enc_hidden", "d_pred_hidden", "d_latent", "d_embedding", "d_ff", "n_head", "d_head"], "train": ["learning_rate", "batch_size", "max_epochs"]}
    config_name = ""
    for key in key_components.keys():
        for component in key_components[key]:
            config_name += "["+str(configs[key][component])+component+"]"
    final_model_path = os.path.join(model_dir, f"[Final_model]{config_name}.ckpt")

configs.train.update({"config_name": config_name,
                        "final_model_path": final_model_path})

# PART 2: Dataset setting

In [3]:
tstart = time.time()
org_config_keys_temp = copy.copy(org_config_keys["data"])
org_config_keys_temp.pop(org_config_keys_temp.index("use_pretrained_tokenizer"))
org_config_keys_temp.pop(org_config_keys_temp.index("data_file"))
dataset_config_name = "-".join([str(key)+"="+str(value) for (key,value) in configs.data.items() if key in org_config_keys_temp])
dataset_path = os.path.join(data_dir, "pickled_dataset", "[tech_dataset]"+dataset_config_name+".pickle")
if os.path.exists(dataset_path) and args.do_save is False:
    print("Load pickled dataset...")
    with open(dataset_path, "rb") as f:
        tech_dataset = pickle.load(f)   # Load pickled dataset if dataset with same configuration already saved
        if tech_dataset.pretrained_enc != configs.data.pretrained_enc or tech_dataset.pretrained_dec != configs.data.pretrained_dec:
                tech_dataset.pretrained_enc = configs.data.pretrained_enc
                tech_dataset.pretrained_dec = configs.data.pretrained_dec
                tech_dataset.tokenizers = tech_dataset.get_tokenizers()
        for tk in tech_dataset.tokenizers.values():
            if "vocab_size" not in dir(tk):
                tk.vocab_size = tk.get_vocab_size()
    print("Pickled dataset loaded")
else:
    print("Make dataset...")
    tech_dataset = TechDataset(configs.data)
    with open(dataset_path, "wb") as f:
        tech_dataset.rawdata = None
        pickle.dump(tech_dataset, f)
tend = time.time()
print(f"{np.round(tend-tstart,4)} sec elapsed for loading patents for class [{configs.data.target_ipc}]")

Make dataset...



Tokenizer is trained and saved
67.3479 sec elapsed for loading patents for class [H01L]


In [4]:
configs.model.update({"tokenizers": tech_dataset.tokenizers,
                    "n_enc_seq_claim": tech_dataset.max_seq_len_claim,
                    "n_dec_seq_claim": tech_dataset.max_seq_len_claim,
                    "n_enc_seq_class": tech_dataset.max_seq_len_class,
                    "n_dec_seq_class": tech_dataset.max_seq_len_class,
                    "n_outputs": 1 if configs.data.pred_type=="regression" else tech_dataset.n_outputs,
                    "i_padding": tech_dataset.tokenizers["class_enc"].token_to_id("<PAD>")})

# PART 3: Training

In [5]:
sampler = CVSampler(tech_dataset, n_folds=configs.train.n_folds, test_ratio=0.1, stratify=True)
cv_idx = sampler.get_idx_dict()
print(f"#Samples\nTrain: {len(cv_idx[0]['train'])}, Validation: {len(cv_idx[0]['val'])}, Test: {len(cv_idx[0]['test'])}")

#Samples
Train: 24389, Validation: 6098, Test: 3388


## PART 3-2: Dataset construction and model training

In [6]:
## Construct datasets
train_idx = cv_idx[0]['train']
val_idx = cv_idx[0]['val']
test_idx = cv_idx[0]['test']
whole_idx = np.concatenate([train_idx, val_idx])

train_dataset = Subset(tech_dataset, train_idx)
val_dataset = Subset(tech_dataset, val_idx)
test_dataset = Subset(tech_dataset, test_idx)
whole_dataset = Subset(tech_dataset, whole_idx)

train_loader = DataLoader(train_dataset, batch_size=configs.train.batch_size, shuffle=True, num_workers=4, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=configs.train.batch_size if len(val_idx)>configs.train.batch_size else len(val_idx), shuffle=True, num_workers=4, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=configs.train.batch_size if len(test_idx)>configs.train.batch_size else len(test_idx), shuffle=False, num_workers=4)
whole_loader = DataLoader(whole_dataset, batch_size=configs.train.batch_size, shuffle=False, num_workers=4)

## Load best model or build model
final_model = build_model(configs.model, tokenizers=tech_dataset.tokenizers)

In [7]:
np.unique(tech_dataset.Y_digitized, return_counts=True)

(array([0, 1]), array([30722,  3153]))

### predictor parameter update 실험, z-sampling 실험

In [8]:
class_weights = torch.tensor(np.unique(tech_dataset.Y[whole_idx], return_counts=True)[1][::-1].copy()).to(device)
class_weights = class_weights / class_weights.sum()
from utils import KLDLoss
from parallel import DataParallel
model_params = configs.model
loss_recon = torch.nn.CrossEntropyLoss(ignore_index=model_params['i_padding'])
loss_y = torch.nn.MSELoss() if model_params['n_outputs']==1 else torch.nn.NLLLoss(weight=class_weights)
loss_kld = KLDLoss()
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, final_model.module.parameters()), lr=configs.train['learning_rate'])

In [20]:
for p in final_model.module.predictor.parameters():
    print(p.requires_grad)

True
True
True
True
True
True


In [21]:
batch_data = next(iter(train_loader))
batch_data = to_device(batch_data, device)

In [22]:
outputs = final_model.module(batch_data["text_inputs"], batch_data["text_outputs"])
preds_recon = outputs["dec_outputs"].permute(0,2,1)
trues_recon = batch_data["text_outputs"]
preds_y = outputs["pred_outputs"]
trues_y = batch_data["targets"]
preds_mu = outputs["mu"]
preds_logvar = outputs["logvar"]

In [23]:
loss = loss_recon(preds_recon, trues_recon) + loss_kld(preds_mu, preds_logvar) + loss_y(preds_y, trues_y)
print(loss)

tensor(277.7591, device='cuda:1', grad_fn=<AddBackward0>)


In [24]:
enc_outputs, z, mu, logvar = final_model.module.encode(batch_data["text_inputs"])
pred_outputs = final_model.module.predictor(z)

In [25]:
z[:10,:10]

tensor([[ 1.3289, -0.8290,  2.1928, -0.8552,  0.4497, -0.4875,  0.3141, -0.3630,
          0.4322,  2.0363],
        [ 0.0231,  0.8366,  0.2612, -0.9064, -0.9936,  0.2772,  0.6317,  0.9932,
         -0.5697, -0.9820],
        [ 1.4624,  0.0575,  1.6061, -0.7062,  0.7521,  0.7862, -0.8314,  1.0175,
          0.3154,  1.2769],
        [-0.3571, -0.2611,  1.9931, -1.2668, -0.0377, -2.1997, -0.8126,  0.8696,
         -0.1465, -0.5589],
        [-0.6340,  0.5706, -1.7299,  0.0265, -0.0048, -1.1392,  1.3421,  1.0491,
         -0.8917, -0.4334],
        [-0.6902, -1.3615,  0.4342, -0.1711,  0.4508, -1.6071, -0.2748,  1.2614,
          0.1740, -0.6839],
        [ 0.0041,  0.0573, -0.5704, -1.4403,  1.4748, -0.4294, -0.9826,  1.9431,
         -1.3444, -0.8872],
        [ 0.0756,  0.2134,  2.2455, -0.3017,  0.6057,  0.0924,  1.3588,  0.4385,
         -1.3261,  1.3466],
        [ 0.8683,  0.3203, -0.9666, -0.1814, -0.0961, -1.0364, -1.0002,  0.6757,
          1.5087,  0.8372],
        [-0.3333,  

In [26]:
z.sum()

tensor(-4.4419, device='cuda:1', grad_fn=<SumBackward0>)

In [27]:
for p in final_model.module.predictor.parameters():
    print(p.sum())
for p in final_model.module.predictor.parameters():
    print(p)

tensor(-3.0592, device='cuda:1', grad_fn=<SumBackward0>)
tensor(-0.0443, device='cuda:1', grad_fn=<SumBackward0>)
tensor(1.8108, device='cuda:1', grad_fn=<SumBackward0>)
tensor(-0.5879, device='cuda:1', grad_fn=<SumBackward0>)
tensor(-0.3179, device='cuda:1', grad_fn=<SumBackward0>)
tensor(0.3017, device='cuda:1', grad_fn=<SumBackward0>)
Parameter containing:
tensor([[ 7.9141e-03, -8.1524e-03,  2.6540e-02,  6.9058e-02, -1.2883e-01,
         -1.0029e-01, -8.4398e-03,  7.1938e-02, -1.2096e-01, -3.7986e-02,
          1.4293e-01,  4.8835e-04, -1.3684e-01,  9.2880e-02, -1.9429e-02,
         -1.2836e-01, -1.8933e-02,  3.7663e-03,  1.2849e-01, -7.3794e-02,
         -7.8411e-02, -1.2944e-01,  3.3521e-02, -1.3669e-01, -1.3464e-02,
         -1.3530e-01, -1.4054e-01, -4.2684e-02,  1.0326e-01, -2.7437e-02,
         -2.1877e-02,  2.2328e-02,  1.4212e-01,  1.7567e-02, -1.1428e-01,
         -3.0918e-02, -1.4953e-02, -1.2023e-01,  8.6323e-03, -1.3624e-01,
          7.5921e-02, -2.6093e-02, -2.2029e-02

In [28]:
loss.backward()
optimizer.step()

In [29]:
outputs = final_model.module(batch_data["text_inputs"], batch_data["text_outputs"])
preds_recon = outputs["dec_outputs"].permute(0,2,1)
trues_recon = batch_data["text_outputs"]
preds_y = outputs["pred_outputs"]
trues_y = batch_data["targets"]
preds_mu = outputs["mu"]
preds_logvar = outputs["logvar"]

In [30]:
loss = loss_recon(preds_recon, trues_recon) + loss_kld(preds_mu, preds_logvar) + loss_y(preds_y, trues_y)
print(loss)

tensor(256.6693, device='cuda:1', grad_fn=<AddBackward0>)


In [31]:
enc_outputs, z, mu, logvar = final_model.module.encode(batch_data["text_inputs"])
pred_outputs = final_model.module.predictor(z)

In [32]:
z[:10,:10]

tensor([[ 1.4736, -0.2272, -0.9766,  0.0770,  0.5192, -0.5376, -1.7196,  1.1690,
          0.6373,  0.6492],
        [ 0.7956,  0.9717,  0.9515, -0.6368,  0.3135, -0.7557,  0.5723,  0.1617,
         -0.6741,  0.0995],
        [ 0.7018,  0.3282,  1.0309, -0.9190,  0.6788, -0.2095, -0.3714,  1.7240,
          0.6228,  0.6429],
        [-1.3614,  0.5443,  0.7771,  0.0175,  0.2961, -1.5920, -1.8737,  1.0537,
          0.2269,  0.0478],
        [ 1.3284,  0.4488, -0.4129, -0.5439,  0.7912, -0.6398, -1.2722,  0.7707,
         -0.0897,  0.3950],
        [-0.1995,  0.4182,  0.7557,  0.2058, -0.5340, -0.5134, -0.2542,  0.0318,
         -0.6629, -1.2671],
        [ 0.1741, -0.1415,  1.5415, -0.5425,  0.8440, -1.1065,  0.3004,  0.8632,
          0.0292, -0.5559],
        [ 0.4454, -0.1149,  2.1670, -0.5932,  0.8308,  0.4289, -1.6735, -1.6040,
          1.8533,  1.0629],
        [ 0.3557, -1.4769,  2.3685, -0.1974,  1.0292, -1.7125, -3.3732,  1.7291,
          0.3478, -0.5429],
        [-0.3506, -

In [33]:
z.sum()

tensor(-32.7386, device='cuda:1', grad_fn=<SumBackward0>)

In [34]:
for p in final_model.module.predictor.parameters():
    print(p.sum())
for p in final_model.module.predictor.parameters():
    print(p)

tensor(-3.0522, device='cuda:1', grad_fn=<SumBackward0>)
tensor(-0.0478, device='cuda:1', grad_fn=<SumBackward0>)
tensor(1.7943, device='cuda:1', grad_fn=<SumBackward0>)
tensor(-0.5874, device='cuda:1', grad_fn=<SumBackward0>)
tensor(-0.3179, device='cuda:1', grad_fn=<SumBackward0>)
tensor(0.3017, device='cuda:1', grad_fn=<SumBackward0>)
Parameter containing:
tensor([[ 7.4140e-03, -7.6523e-03,  2.6040e-02,  6.9558e-02, -1.2833e-01,
         -9.9794e-02, -7.9398e-03,  7.1437e-02, -1.2046e-01, -3.7486e-02,
          1.4243e-01,  9.8834e-04, -1.3634e-01,  9.3380e-02, -1.9929e-02,
         -1.2786e-01, -1.8433e-02,  4.2662e-03,  1.2799e-01, -7.4294e-02,
         -7.8911e-02, -1.2994e-01,  3.3020e-02, -1.3719e-01, -1.2964e-02,
         -1.3480e-01, -1.4104e-01, -4.2184e-02,  1.0376e-01, -2.7937e-02,
         -2.1377e-02,  2.2828e-02,  1.4262e-01,  1.7067e-02, -1.1378e-01,
         -3.0418e-02, -1.4453e-02, -1.2073e-01,  8.1322e-03, -1.3574e-01,
          7.5421e-02, -2.6592e-02, -2.1529e-02

In [44]:
final_model.module.freeze(module_name="predictor")
for p in final_model.module.predictor.parameters():
    print(p.requires_grad)

False
False
False
False
False
False


In [45]:
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, final_model.module.parameters()), lr=configs.train['learning_rate'])

In [55]:
loss.backward()
optimizer.step()

In [56]:
outputs = final_model.module(batch_data["text_inputs"], batch_data["text_outputs"])
preds_recon = outputs["dec_outputs"].permute(0,2,1)
trues_recon = batch_data["text_outputs"]
preds_y = outputs["pred_outputs"]
trues_y = batch_data["targets"]
preds_mu = outputs["mu"]
preds_logvar = outputs["logvar"]

In [57]:
loss = loss_recon(preds_recon, trues_recon) + loss_kld(preds_mu, preds_logvar) + loss_y(preds_y, trues_y)
print(loss)

tensor(207.7741, device='cuda:1', grad_fn=<AddBackward0>)


In [58]:
enc_outputs, z, mu, logvar = final_model.module.encode(batch_data["text_inputs"])
pred_outputs = final_model.module.predictor(z)

In [59]:
z[:10,:10]

tensor([[-3.5709e-01, -4.9432e-01, -8.0583e-01, -4.5443e-01, -1.2907e-01,
          1.6342e-01,  2.6257e-01,  4.0090e-01,  1.0620e+00, -2.1962e-01],
        [ 8.7164e-01,  1.3203e-01,  9.4832e-01,  8.6002e-02, -2.2500e-01,
         -5.3908e-01, -1.5957e+00, -4.6577e-01, -8.0176e-01,  1.8494e+00],
        [-1.0795e-01, -2.0285e+00,  1.2273e+00, -1.4995e-01,  4.4946e-01,
         -7.1788e-01, -1.5552e-01,  7.4372e-01, -5.1485e-01,  3.7813e-01],
        [ 4.8809e-01,  5.6596e-01,  1.0377e+00, -3.9848e-01, -5.3214e-01,
         -6.4806e-01,  6.9967e-01,  1.3287e+00,  8.6062e-01,  4.3469e-01],
        [ 7.0350e-01,  1.3099e+00,  3.5010e-01, -1.1539e+00,  2.0622e-01,
          1.1206e+00,  1.0781e+00, -2.3437e-01, -8.5554e-01,  2.1002e-01],
        [-1.1574e-01,  4.0096e-01,  2.3872e+00, -5.4562e-01, -4.1038e-01,
          5.1674e-01,  2.4054e-01,  2.3169e+00,  7.0590e-01,  1.4846e+00],
        [-2.7840e-01,  1.7006e-02,  4.1294e-01, -1.0329e+00,  2.2243e-01,
          1.1384e-01, -4.3268e-0

In [60]:
z.sum()

tensor(65.1528, device='cuda:1', grad_fn=<SumBackward0>)

In [52]:
for p in final_model.module.predictor.parameters():
    print(p.sum(), p.requires_grad)
for p in final_model.module.predictor.parameters():
    print(p)

tensor(-3.0452, device='cuda:1') False
tensor(-0.0513, device='cuda:1') False
tensor(1.7778, device='cuda:1') False
tensor(-0.5869, device='cuda:1') False
tensor(-0.3179, device='cuda:1') False
tensor(0.3017, device='cuda:1') False
Parameter containing:
tensor([[ 6.9140e-03, -7.1523e-03,  2.5539e-02,  7.0057e-02, -1.2782e-01,
         -9.9294e-02, -7.4398e-03,  7.0937e-02, -1.1996e-01, -3.6986e-02,
          1.4193e-01,  1.4883e-03, -1.3584e-01,  9.3879e-02, -2.0429e-02,
         -1.2736e-01, -1.7933e-02,  4.7662e-03,  1.2749e-01, -7.4794e-02,
         -7.9410e-02, -1.3044e-01,  3.2520e-02, -1.3769e-01, -1.2464e-02,
         -1.3430e-01, -1.4154e-01, -4.1684e-02,  1.0426e-01, -2.8437e-02,
         -2.0877e-02,  2.3328e-02,  1.4312e-01,  1.6566e-02, -1.1328e-01,
         -2.9918e-02, -1.3953e-02, -1.2123e-01,  7.6322e-03, -1.3524e-01,
          7.4920e-02, -2.7092e-02, -2.1028e-02,  5.6670e-02,  3.6748e-03,
         -4.5485e-02,  8.7172e-02, -6.6364e-02],
        [ 8.4926e-02, -7.4478e-

In [61]:
for p in final_model.module.predictor.parameters():
    print(p.sum(), p.requires_grad)
for p in final_model.module.predictor.parameters():
    print(p)

tensor(-3.0452, device='cuda:1') False
tensor(-0.0513, device='cuda:1') False
tensor(1.7778, device='cuda:1') False
tensor(-0.5869, device='cuda:1') False
tensor(-0.3179, device='cuda:1') False
tensor(0.3017, device='cuda:1') False
Parameter containing:
tensor([[ 6.9140e-03, -7.1523e-03,  2.5539e-02,  7.0057e-02, -1.2782e-01,
         -9.9294e-02, -7.4398e-03,  7.0937e-02, -1.1996e-01, -3.6986e-02,
          1.4193e-01,  1.4883e-03, -1.3584e-01,  9.3879e-02, -2.0429e-02,
         -1.2736e-01, -1.7933e-02,  4.7662e-03,  1.2749e-01, -7.4794e-02,
         -7.9410e-02, -1.3044e-01,  3.2520e-02, -1.3769e-01, -1.2464e-02,
         -1.3430e-01, -1.4154e-01, -4.1684e-02,  1.0426e-01, -2.8437e-02,
         -2.0877e-02,  2.3328e-02,  1.4312e-01,  1.6566e-02, -1.1328e-01,
         -2.9918e-02, -1.3953e-02, -1.2123e-01,  7.6322e-03, -1.3524e-01,
          7.4920e-02, -2.7092e-02, -2.1028e-02,  5.6670e-02,  3.6748e-03,
         -4.5485e-02,  8.7172e-02, -6.6364e-02],
        [ 8.4926e-02, -7.4478e-

In [79]:
final_model.module.freeze(module_name="predictor", defreeze=True)
for p in final_model.module.predictor.parameters():
    print(p.requires_grad)

True
True
True
True
True
True


In [78]:
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, final_model.module.parameters()), lr=configs.train['learning_rate'])

In [80]:
loss.backward()
optimizer.step()

In [81]:
outputs = final_model.module(batch_data["text_inputs"], batch_data["text_outputs"])
preds_recon = outputs["dec_outputs"].permute(0,2,1)
trues_recon = batch_data["text_outputs"]
preds_y = outputs["pred_outputs"]
trues_y = batch_data["targets"]
preds_mu = outputs["mu"]
preds_logvar = outputs["logvar"]

In [82]:
loss = loss_recon(preds_recon, trues_recon) + loss_kld(preds_mu, preds_logvar) + loss_y(preds_y, trues_y)
print(loss)

tensor(163.2842, device='cuda:1', grad_fn=<AddBackward0>)


In [83]:
enc_outputs, z, mu, logvar = final_model.module.encode(batch_data["text_inputs"])
pred_outputs = final_model.module.predictor(z)

In [84]:
z[:10,:10]

tensor([[-0.9188,  1.0647,  0.3373, -1.0589, -0.4356,  1.0034, -0.1101,  0.3827,
          0.9371, -1.9254],
        [ 0.0978,  0.7434,  0.6444, -0.2944,  0.3274,  0.8281, -0.6298, -0.8488,
         -1.1199,  0.7339],
        [ 0.1135, -0.1256,  1.8128,  0.0260, -0.9463, -1.1076, -0.8307,  0.4789,
          1.0825,  0.2068],
        [-1.1163,  0.1125,  1.3946, -0.4888,  0.0986, -0.4214, -0.6237,  0.1326,
         -1.5205, -0.6777],
        [-0.9647, -0.5700, -0.3964,  0.4473,  0.7972, -1.0017,  0.0936,  0.8673,
         -0.1744,  0.2031],
        [-0.2352,  0.5598,  0.3131, -0.2481,  1.2925,  0.4672, -1.5914,  0.9305,
          1.9326,  0.0876],
        [-0.9283, -1.5283,  1.0758, -0.7223,  1.3624,  0.7709,  0.9271,  1.9560,
          1.0451, -1.0441],
        [-2.0542,  0.6931,  1.5115,  0.0730,  1.6530,  1.1155, -1.9170,  0.8677,
         -2.0559,  1.5885],
        [-0.7855, -1.5159,  0.9893, -0.8754,  0.3694, -0.4529, -0.2145, -0.4598,
         -0.6518,  0.2534],
        [ 1.7088,  

In [85]:
z.sum()

tensor(-20.0368, device='cuda:1', grad_fn=<SumBackward0>)

In [86]:
for p in final_model.module.predictor.parameters():
    print(p.sum(), p.requires_grad)
for p in final_model.module.predictor.parameters():
    print(p)

tensor(-3.0292, device='cuda:1', grad_fn=<SumBackward0>) True
tensor(-0.0533, device='cuda:1', grad_fn=<SumBackward0>) True
tensor(1.7583, device='cuda:1', grad_fn=<SumBackward0>) True
tensor(-0.5889, device='cuda:1', grad_fn=<SumBackward0>) True
tensor(-0.3179, device='cuda:1', grad_fn=<SumBackward0>) True
tensor(0.3017, device='cuda:1', grad_fn=<SumBackward0>) True
Parameter containing:
tensor([[ 0.0064, -0.0067,  0.0250,  0.0706, -0.1283, -0.0988, -0.0069,  0.0704,
         -0.1195, -0.0375,  0.1414,  0.0020, -0.1353,  0.0944, -0.0209, -0.1269,
         -0.0174,  0.0053,  0.1270, -0.0753, -0.0789, -0.1309,  0.0320, -0.1382,
         -0.0120, -0.1338, -0.1420, -0.0412,  0.1048, -0.0289, -0.0204,  0.0228,
          0.1426,  0.0171, -0.1128, -0.0294, -0.0135, -0.1217,  0.0071, -0.1347,
          0.0744, -0.0276, -0.0205,  0.0562,  0.0032, -0.0450,  0.0877, -0.0659],
        [ 0.0844, -0.0740,  0.0745, -0.0539, -0.0826, -0.0302, -0.1396, -0.1312,
          0.0010,  0.0545, -0.0629,  0.1

## PART 3-3: Training evaluation

In [ ]:
if args.eval_train_set:
    ## Evaluation on train dataset
    print("Validate model on train dataset")
    # trues_recon_train, preds_recon_train, trues_y_train, preds_y_train = validate_model(final_model, whole_loader, configs.model, configs.train)
    val_res_train = validate_model_mp(final_model, whole_dataset, mp=mp, model_params=configs.model, train_params=configs.train)
    trues_recon_train = np.concatenate([res["recon"]["true"] for res in val_res_train.values()])
    preds_recon_train = np.concatenate([res["recon"]["pred"] for res in val_res_train.values()])
    trues_y_train = np.concatenate([res["y"]["true"] for res in val_res_train.values()])
    preds_y_train = np.concatenate([res["y"]["pred"] for res in val_res_train.values()])

    eval_recon_train = perf_eval("TRAIN_SET", trues_recon_train, preds_recon_train, configs=configs, pred_type='generative', tokenizer=final_model.module.tokenizer)
    eval_recon_train = perf_eval("TRAIN_SET", trues_recon_train, preds_recon_train, configs=configs, pred_type='generative', tokenizer=final_model.module.tokenizer)
    eval_y_train = perf_eval("TRAIN_SET", trues_y_train, preds_y_train, configs=configs, pred_type=configs.data.pred_type)
    if configs.data.pred_type == "classification":
        eval_y_train, confmat_y_train = eval_y_train
else:
    eval_recon_train = eval_y_train = None

## Evaluation on test dataset
print("Validate model on test dataset")
# trues_recon_test, preds_recon_test, trues_y_test, preds_y_test = validate_model(final_model, test_loader, configs.model, configs.train)
val_res_test = validate_model_mp(final_model, test_dataset, mp=mp, batch_size=64, model_params=configs.model, train_params=configs.train)
trues_recon_test = np.concatenate([res["recon"]["true"] for res in val_res_test.values()])
preds_recon_test = np.concatenate([res["recon"]["pred"] for res in val_res_test.values()])
trues_y_test = np.concatenate([res["y"]["true"] for res in val_res_test.values()])
preds_y_test = np.concatenate([res["y"]["pred"] for res in val_res_test.values()])

eval_recon_test = perf_eval("TEST_SET", trues_recon_test, preds_recon_test, configs=configs,  pred_type='generative', tokenizer=final_model.module.tokenizer)
eval_y_test = perf_eval("TEST_SET", trues_y_test, preds_y_test, configs=configs, pred_type=configs.data.pred_type)
if configs.data.pred_type == "classification":
    eval_y_test, confmat_y_test = eval_y_test

eval_recon_res = pd.concat([eval_recon_train, eval_recon_test], axis=0)
eval_y_res = pd.concat([eval_y_train, eval_y_test], axis=0)
if configs.data.pred_type == "classification":
    confmat_y_res = pd.concat([confmat_y_train, confmat_y_test], axis=0)
